# PRÁCTICA 1
## Estrategias de particionado, Naive Bayes y Análisis ROC
### Fundamentos de Aprendizaje Automático
##### Grupo 1461: Gloria del Valle Cano y Eva Gutiérrez Vanmoen

In [1]:
from abc import ABCMeta, abstractmethod
import random

from functools import reduce
import math
import operator

from Datos import Datos
import Clasificador
import EstrategiaParticionado
import numpy as np
import pandas as pd

from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, roc_curve, auc

import matplotlib.pyplot as plt

from IPython.display import HTML, display

### Apartado 1 : Particionado
Análisis de las dos estrategias de particionado propuestas: simple, y
cruzada, para los conjuntos propuestos: lenses, german y tic-tac-toe. El
análisis consiste en una descripción de los índices de train y test
devueltos por cada uno de los métodos de particionado, junto con un
comentario sobre las ventajas/desventajas de cada uno de ellos.

In [2]:
d1 = Datos('./ConjuntosDatos/tic-tac-toe.data')
d2 = Datos('./ConjuntosDatos/german.data')
d3 = Datos('./ConjuntosDatos/lenses.data')

Hemos cargado los conjuntos de datos que vamos a utilizar y a continuación vamos a mostrar la validación simple implementada.
##### VSimple

In [3]:
 def creaParticiones(self,datos,seed=None):
    
    random.seed(seed)

    # K iteraciones = numero de particiones
    for k in range(self.numeroParticiones):

      p = Particion()

      # Obtenemos una array aleatoria de las filas de nuestros datos
      npaux = np.random.permutation(datos.shape[0])

      i = int(len(npaux.tolist())*self.porcentaje) # indice del porcentaje deseado para entrenamiento
      
      # Dividimos datos en entrenamiento y de prueba
      p.indicesTrain = npaux.tolist()[0:i]
      p.indicesTest = npaux.tolist()[i:]

      self.listaParticiones.append(p) # Insertamos la particion creada

Realizamos una partición y seleccionamos al azar el conjunto de entrenamiento y el resto será para el test.
Lo bueno es que es muy fácil de implementar, pero existe el problema de que la división no puede ser suficientemente determinante como para tomar una decisión, es decir, los índices de test que se cogen para probar el entrenamiento pueden no ser determinantes, demasiados, pocos... Debido a la variabilidad y a la división de los datos, no sabremos qué será más óptimo hasta que hagamos más pruebas.
##### Vcruzada

In [4]:
  def creaParticiones(self,datos,seed=None):   
    
    random.seed(seed)

    # Obtenemos una array aleatoria de las filas de nuestros datos
    npaux1 = np.random.permutation(datos.shape[0])

    nL = int(len(npaux1.tolist())) # numero de lineas de datos

    step = int(nL/self.numeroParticiones) # numero de datos por cada subconjunto

    # laux es una lista de la array aleatoria (npaux2) de nuestros indices de los subconjuntos
    npaux2 = np.random.permutation(range(0,nL,step))
    laux = npaux2.tolist()
    
    # Dividimos en K-1 subconjuntos para entrenamiento y el resto test
    for k in range(self.numeroParticiones):
      
      p = Particion()

      # Primero obtenemos el subconjunto para tests
      i = laux[0] # el indice inicial de nuestros datos de test
      p.indicesTest = npaux1.tolist()[i:i+step]
      laux.pop(0) # eliminamos el indice de bloque ya usado para test

      # K-1 subconjuntos para datos de entrenamiento
      for j in npaux2.tolist():
        if j != i:
          # Si tratamos con el ultimo bloque de datos
          if j == max(npaux2.tolist()):
            p.indicesTrain.extend(npaux1.tolist()[j:])
          else:
            p.indicesTrain.extend(npaux1.tolist()[j:j+step])

      # Insertamos la particion creada de cada iteracion
      self.listaParticiones.append(p)

En este caso, el entrenamiento se crea con nFolds - 1 (nFolds = n divisiones) y el de test con la restante. Es una de las maneras más completas de probar los datos, pero es más costoso a nivel computacional.
###### Indices devueltos por tic-tac-toe.data

In [5]:
NB = Clasificador.ClasificadorNaiveBayes()
print ("VALIDACION SIMPLE (tic-tac-toe.data): ")
v_simple = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',3,0.8)
p = v_simple.creaParticiones(d1.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)

print ("VALIDACION CRUZADA (tic-tac-toe.data): ")
v_cross = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
p = v_cross.creaParticiones(d1.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)
	print ("----")
print ("-----------------------")

VALIDACION SIMPLE (tic-tac-toe.data): 
Indices Test: 
[214, 590, 115, 559, 611, 921, 292, 133, 908, 520, 325, 829, 225, 474, 354, 876, 731, 774, 830, 825, 4, 289, 691, 87, 605, 606, 722, 540, 630, 898, 580, 430, 168, 853, 158, 403, 221, 277, 237, 462, 480, 431, 201, 88, 156, 34, 851, 428, 637, 586, 394, 792, 89, 355, 432, 425, 64, 177, 647, 60, 801, 293, 751, 954, 753, 53, 503, 868, 927, 747, 587, 154, 174, 643, 906, 451, 450, 593, 786, 935, 401, 929, 453, 460, 369, 556, 632, 555, 869, 950, 209, 385, 127, 352, 371, 397, 525, 465, 170, 463, 381, 135, 488, 83, 642, 296, 941, 347, 223, 491, 631, 704, 113, 697, 926, 891, 685, 100, 836, 153, 844, 259, 748, 228, 682, 730, 583, 936, 848, 58, 613, 15, 19, 69, 205, 714, 771, 380, 444, 358, 568, 179, 690, 219, 754, 18, 109, 318, 189, 193, 98, 951, 604, 687, 834, 91, 8, 310, 11, 51, 711, 940, 386, 702, 871, 820, 36, 709, 576, 231, 114, 893, 641, 366, 365, 56, 299, 145, 298, 184, 467, 627, 789, 408, 181, 763, 235, 915, 819, 617, 372, 108]
Indices 

##### Indices devueltos por german.data

In [6]:
NB = Clasificador.ClasificadorNaiveBayes()
print ("VALIDACION SIMPLE (german.data): ")
v_simple = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',3,0.8)
p = v_simple.creaParticiones(d2.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)

print ("VALIDACION CRUZADA (german.data): ")
v_cross = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
p = v_cross.creaParticiones(d2.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)
	print ("----")
print ("-----------------------")

VALIDACION SIMPLE (german.data): 
Indices Test: 
[759, 43, 535, 991, 384, 669, 397, 512, 50, 265, 647, 134, 696, 792, 804, 55, 358, 171, 154, 411, 593, 235, 790, 163, 170, 321, 748, 909, 667, 152, 674, 188, 933, 225, 606, 470, 494, 550, 563, 324, 997, 775, 857, 306, 51, 818, 511, 282, 262, 223, 313, 6, 0, 189, 371, 267, 599, 871, 724, 246, 988, 9, 136, 727, 86, 432, 39, 652, 755, 764, 326, 493, 676, 44, 989, 774, 929, 837, 309, 627, 318, 251, 830, 365, 361, 151, 916, 474, 210, 788, 396, 57, 454, 789, 22, 897, 609, 18, 35, 27, 839, 62, 425, 746, 495, 448, 30, 968, 355, 121, 990, 247, 936, 498, 518, 761, 277, 331, 881, 412, 108, 344, 651, 112, 480, 711, 982, 497, 414, 77, 172, 932, 217, 430, 208, 268, 597, 167, 280, 145, 851, 717, 60, 520, 177, 375, 635, 677, 79, 441, 692, 54, 257, 673, 642, 501, 135, 847, 919, 575, 360, 722, 777, 680, 993, 698, 368, 704, 975, 882, 467, 124, 894, 369, 4, 288, 141, 678, 573, 980, 751, 832, 703, 732, 650, 300, 758, 157, 104, 634, 543, 712, 258, 483, 619, 4

##### Indices devueltos por lenses.data

In [7]:
NB = Clasificador.ClasificadorNaiveBayes()
print ("VALIDACION SIMPLE (lenses.data): ")
v_simple = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',3,0.8)
p = v_simple.creaParticiones(d3.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)

print ("VALIDACION CRUZADA (lenses.data): ")
v_cross = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
p = v_cross.creaParticiones(d3.datos)
for k in p:
	print ("Indices Test: ")
	print (k.indicesTest)
	print ("Indices Train: ")
	print (k.indicesTrain)
	print ("----")
print ("-----------------------")

VALIDACION SIMPLE (lenses.data): 
Indices Test: 
[4, 12, 3, 23, 6]
Indices Train: 
[2, 17, 5, 20, 8, 1, 13, 21, 15, 18, 7, 19, 0, 9, 22, 11, 10, 14, 16]
Indices Test: 
[6, 17, 4, 22, 14]
Indices Train: 
[19, 1, 13, 3, 0, 2, 9, 11, 8, 21, 12, 7, 15, 23, 18, 16, 10, 5, 20]
Indices Test: 
[7, 6, 2, 21, 14]
Indices Train: 
[9, 17, 19, 8, 16, 13, 18, 12, 5, 4, 10, 22, 3, 23, 0, 11, 1, 15, 20]
VALIDACION CRUZADA (lenses.data): 
Indices Test: 
[9, 19, 18, 12]
Indices Train: 
[1, 16, 10, 21, 2, 6, 17, 14, 7, 13, 4, 23, 0, 3, 22, 20, 11, 8, 15, 5]
----
Indices Test: 
[1, 16, 10, 21]
Indices Train: 
[9, 19, 18, 12, 2, 6, 17, 14, 7, 13, 4, 23, 0, 3, 22, 20, 11, 8, 15, 5]
----
Indices Test: 
[2, 6, 17, 14]
Indices Train: 
[9, 19, 18, 12, 1, 16, 10, 21, 7, 13, 4, 23, 0, 3, 22, 20, 11, 8, 15, 5]
----
Indices Test: 
[7, 13, 4, 23]
Indices Train: 
[9, 19, 18, 12, 1, 16, 10, 21, 2, 6, 17, 14, 0, 3, 22, 20, 11, 8, 15, 5]
----
Indices Test: 
[0, 3, 22, 20]
Indices Train: 
[9, 19, 18, 12, 1, 16, 10, 21, 2

### Apartado 2: Naive-Bayes
Tabla con los resultados de la ejecución para los conjuntos de datos
analizados (lenses, tic-tac-toe y german). Considerar los dos tipos de
particionado.
Los resultados se refieren a las tasas de error/acierto y deben incluirse
tanto con la corrección de Laplace como sin ella. Se debe incluir tanto
el promedio de error para las diferentes particiones como su desviación
típica. Es importante mostrar todos los resultados agrupados en una 
tabla para facilitar su evaluación.
Breve análisis de los resultados anteriores

#### Test con tic-tac-toe.data

Probamos primero Validación Simple sin laPlace

In [14]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE11,pred = clasificadorNB.validacion(estrategia, d1, clasificadorNB)
print(MSE11)

[0.3229166666666667]


Probamos de la misma manera con la correccion de laPlace

In [15]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE12,pred = clasificadorNB.validacion(estrategia, d1, clasificadorNB,laPlace=True)
print(MSE12)

[0.6927083333333334]


Ahora con Validación Cruzada sin laPlace

In [16]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE,pred = clasificadorNB.validacion(estrategia, d1, clasificadorNB)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m11 = np.mean(MSE)
print(m11)
print("\nDesviación típica: ")
dt11 = np.std(MSE)
print(dt11)

Lista de MSE:
0.3298429319371728
0.3717277486910995
0.3333333333333333
0.3507853403141361
0.3403141361256545

Media MSE: 
0.3452006980802792

Desviación típica: 
0.015069482125600627


Probamos de la misma manera con la correccion de laPlace

In [17]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE,pred = clasificadorNB.validacion(estrategia, d1, clasificadorNB, laPlace=True)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m12 = np.mean(MSE)
print(m12)
print("\nDesviación típica: ")
dt12 = np.std(MSE)
print(dt12)

Lista de MSE:
0.33507853403141363
0.3403141361256545
0.35602094240837695
0.3333333333333333
0.35602094240837695

Media MSE: 
0.344153577661431

Desviación típica: 
0.00995835435789675


#### Test con german.data

Probamos primero Validación Simple sin laPlace

In [18]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE21,pred = clasificadorNB.validacion(estrategia, d2, clasificadorNB)
print(MSE21)

KeyError: None

Probamos de la misma manera con la correccion de laPlace

In [19]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE22,pred = clasificadorNB.validacion(estrategia, d2, clasificadorNB,laPlace=True)
print(MSE22)

KeyError: None

Ahora con Validación Cruzada sin laPlace

In [20]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE,pred = clasificadorNB.validacion(estrategia, d2, clasificadorNB)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m21 = np.mean(MSE)
print(m21)
print("\nDesviación típica: ")
dt21 = np.std(MSE)
print(dt21)

KeyError: None

Probamos de la misma manera con la correccion de laPlace

In [21]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE,pred = clasificadorNB.validacion(estrategia, d2, clasificadorNB, laPlace=True)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m22 = np.mean(MSE)
print(m21)
print("\nDesviación típica: ")
dt22 = np.std(MSE)
print(dt22)

KeyError: None

NO ENTIENDO POR QUE, EN EL MAIN SALE BIEN (ver salida del main)
__________________________________________
SIN LAPLACE:
Error NB simple: 
0.695

Error NB cruzada: 
[0.75, 0.275, 0.345, 0.31, 0.32]

Media errores NB cruzada: 
0.4

__________________________________________
CON LAPLACE:
Error NB simple: 
0.32

Error NB cruzada: 
[0.31, 0.295, 0.29, 0.325, 0.28]

Media errores NB cruzada: 
0.3

In [22]:
MSE21 = 0.695
MSE22 = 0.32
m21 = 0.4
m22 = 0.3
l1 = [0.75, 0.275, 0.345, 0.31, 0.32]
l2 = [0.31, 0.295, 0.29, 0.325, 0.28]
dt21 = np.std(l1)
dt22 = np.std(l2)

#### Test con lenses.data

Probamos primero Validación Simple sin laPlace

In [23]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE31,pred = clasificadorNB.validacion(estrategia, d3, clasificadorNB)
print(MSE31)

[0.6]


Probamos de la misma manera con la corrección de laPlace

In [24]:
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE32,pred = clasificadorNB.validacion(estrategia, d3, clasificadorNB, laPlace=True)
print(MSE32)

[1.0]


Ahora con Validación Cruzada sin laPlace

In [25]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE,pred = clasificadorNB.validacion(estrategia, d3, clasificadorNB)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m31 = np.mean(MSE)
print(m31)
print("\nDesviación típica: ")
dt31 = np.std(MSE)
print(dt31)

Lista de MSE:
0.75
0.75
0.5
1.0
0.75

Media MSE: 
0.75

Desviación típica: 
0.15811388300841897


Probamos de la misma manera con la corrección de laPlace

In [26]:
estrategia = EstrategiaParticionado.ValidacionCruzada('ValidacionCruzada',5)
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
MSE,pred = clasificadorNB.validacion(estrategia, d3, clasificadorNB, laPlace=True)
print("Lista de MSE:")
for i in MSE:
    print (i)
print("\nMedia MSE: ")
m32 = np.mean(MSE)
print(m32)
print("\nDesviación típica: ")
dt32 = np.std(MSE)
print(dt32)

Lista de MSE:
0.5
0.0
0.5
0.25
0.75

Media MSE: 
0.4

Desviación típica: 
0.25495097567963926


#### Resumen en tablas

In [28]:
salida = "<table><tr><th>VSimple</th><th>tic-tac-toe.data</th><th>german.data</th><th>lenses.data</th></tr><tr><td>NB sin laPlace</td>"
salida += "<td>%f</td>" % (MSE11[0])

#salida += "<td>%f</td>" % (MSE21[0])
salida += "<td>%f</td>" % (MSE21)

salida += "<td>%f</td></tr>" % (MSE31[0])
salida += "<tr><td>NB con laPlace</td>"
salida += "<td>%f</td>" % (MSE12[0])

#salida += "<td>%f</td>" % (MSE22[0])
salida += "<td>%f</td>" % (MSE22)

salida += "<td>%f</td></tr></table>" % (MSE32[0])

salida2 = "<table><tr><th>VCruzada</th><th>tic-tac-toe.data</th><th>german.data</th><th>lenses.data</th></tr><tr><td>NB sin laPlace</td>"
salida2 += "<td>%.4f ± %.3f</td>" % (m11,dt11)

salida2 += "<td>%.4f ± %.3f</td>" % (m21,dt21)
salida2 += "<td>%.4f ± %.3f</td>" % (m31,dt31)
salida2 += "<tr><td>NB con Laplace</td>"
salida2 += "<td>%.4f ± %.3f</td>" % (m12,dt12)
salida2 += "<td>%.4f ± %.3f</td>" % (m22,dt22)
salida2 += "<td>%.4f ± %.3f</td>" % (m32,dt32)

In [29]:
display(HTML(salida))

VSimple,tic-tac-toe.data,german.data,lenses.data
NB sin laPlace,0.322917,0.695000,0.600000
NB con laPlace,0.692708,0.320000,1.000000


In [30]:
display(HTML(salida2))

VCruzada,tic-tac-toe.data,german.data,lenses.data
NB sin laPlace,0.3452 ± 0.015,0.4000 ± 0.176,0.7500 ± 0.158
NB con Laplace,0.3442 ± 0.010,0.3000 ± 0.016,0.4000 ± 0.255


Parece que podemos comprobar lo que intuíamos, y es que obtendremos un error menor utilizando validación cruzada.

### Apartado 3: Scikit-Learn
Incluir los mismos resultados que en el apartado 2 pero usando los
métodos del paquete scikit-learn. Comparar y analizar los resultados. 

##### La implementación para estos resultados se encuentra en el módulo ParticionadoSK.py


###### Para tic-tac-toe.data
* MSE de NB usando VSimple con laPlace: 0.2956506610995129 con dt: 0.026530565154327092
* MSE de NB usando VCruzada con laPlace: 0.5490605427974948 con dt: 0.11482254697286015
* MSE de NB usando VSimple sin laPlace: 0.2961986778009742 con dt: 0.026569633972322836
* MSE de NB usando VCruzada sin laPlace: 0.5490605427974948 con dt: 0.11482254697286015

###### Para german.data
* MSE de NB usando VSimple con laPlace: 0.3649080000000001 con dt: 0.02825695553310726
* MSE de NB usando VCruzada con laPlace: 0.361 con dt: 0.025000000000000022
* MSE de NB usando VSimple sin laPlace: 0.364556 con dt: 0.028259845434821487
* MSE de NB usando VCruzada sin laPlace: 0.362 con dt: 0.02400000000000002

###### Para lenses.data
* MSE de NB usando VSimple con laPlace: 0.3819444444444444 con dt: 0.16996164236833913
* MSE de NB usando VCruzada con laPlace: 0.25174825174825166 con dt: 0.02097902097902099
* MSE de NB usando VSimple sin laPlace: 0.35416666666666663 con dt: 0.18827002377397167
* MSE de NB usando VCruzada sin laPlace: 0.25174825174825166 con dt: 0.02097902097902099

Podemos ver que la implementación de Scikit sigue siendo más óptima que la propuesta, pero se acercan bastante y nos ha servido para entender el funcionamiento y la importancia de la validación.

Repetimos el código (un poco más simplificado) más abajo a modo de pseudocódigo:

### Apartado 4: Evaluación de hipótesis mediante Análisis ROC
Matriz de confusión y diagramas del clasificador en el espacio ROC:

No hemos conseguido obtener los datos... Sabemos cómo hacer el análisis ROC, lo implementamos más abajo.

#### Matriz de confusión: tic-tac-toe.data

In [31]:
actual = pd.Series([MSE11[0], MSE12[0]], name='Actual')
pred = pd.Series([m11, m12], name='Prediccion')
df_confusion = pd.crosstab(actual,pred)
print (df_confusion) # NO ES

Prediccion  0.344154  0.345201
Actual                        
0.322917           0         1
0.692708           1         0


In [32]:
actual = pd.Series([MSE31[0], MSE32[0]], name='Actual')
pred = pd.Series([m31, m32], name='Prediccion')
df_confusion = pd.crosstab(actual,pred)
print (df_confusion) # NO ES

Prediccion  0.40  0.75
Actual                
0.6            0     1
1.0            1     0


In [82]:
clases = list(d1.datos[:,-1].flatten().tolist())
clasificadorNB = Clasificador.ClasificadorNaiveBayes()
estrategia = EstrategiaParticionado.ValidacionSimple('ValidacionSimple',5,0.8)
err, pred = clasificadorNB.validacion(estrategia, d1, clasificadorNB)
df_confusion = confusion_matrix(clases, pred)
#sabemos que clases y pred tienen que ser del mismo tamaño

ValueError: Found input variables with inconsistent numbers of samples: [958, 192]

In [52]:
punt = df_confusion.ravel()
tnr, fpr, fnr, tpr = punt/np.sum(punt)
print("TNR: " + tnr)
print("FPR: " + fpr)
print("FNR: " + fnr)
print("TPR: " + tpr)

AttributeError: 'DataFrame' object has no attribute 'ravel'

In [ ]:
fpr, tpr, t = metrics.roc_curve(clases,pred)
roc_auc = metrics.auc(tpr,fpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Curva ROC (area=%0.2f)' %roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("FPR")
plt.ylabel("TPR")
plt.title("Resultado ROC")
plt.legend(loc='lower right')
plt.show()